In [ ]:
# 🧱 Agent: Analyze and generate signals based on the fetched data for multiple tokens, considering sentiment-weighted total
def analyze_multiple_tokens_data_with_sentiment(all_tokens_data):
    # Dictionary to store processed data for each token
    all_tokens_analysis = {}

    # Loop through each token's data
    for token, combined_df in all_tokens_data.items():
        print(f"Processing data for {token}...")

        # Calculate daily returns based on the 'value_y' column (price data)
        combined_df['return'] = combined_df['value_y'].pct_change() * 100

        # Identify social volume spikes using 'value_x' for social volume
        combined_df['rolling_avg'] = combined_df['value_x'].rolling(window=7).mean()

        # Buy signal: when social volume spikes above 1.5x the rolling average and sentiment-weighted total is positive
        combined_df['buy_signal'] = (combined_df['value_x'] > (combined_df['rolling_avg'] * 1.5)) & (combined_df['value'] > 0)

        # Sell signal: when social volume drops below the rolling average, return is negative, and sentiment-weighted total is negative
        combined_df['sell_signal'] = (combined_df['value_x'] < combined_df['rolling_avg']) & (combined_df['return'] < 0) & (combined_df['value'] < 0)

        # Drop rows with missing values (NaN)
        combined_df = combined_df.dropna()
        combined_df.reset_index(inplace=True)  # This will turn the index into a regular column

        # Store the processed data for this token
        all_tokens_analysis[token] = combined_df

    return all_tokens_analysis
